#GoProtect_2

### **Заказчик**

- Компания GoProtect

### **Описание проекта**
Сервис  “Мой Чемпион” помогает спортивным школам фигурного катания, тренерам мониторить результаты своих подопечных и планировать дальнейшее развитие спортсменов.

**Цель:**

Создать решение для стандартизации названий спортивных школ.
Например одна и та же школа может быть записана по разному


Надо сопоставить эти варианты эталонному названию из предоставленной таблицы


**Задачи:**

- Изучить данные – эталонные названия СШ и варианты пользовательского ввода
Подготовить обучающий набор данных на основе эталонного датасета

- Создать модель для подбора наиболее вероятных названий при ошибочном вводе

- Создать функцию (класс, модуль) для применения в сервисе

 - возможность выбора количества кандидатов
 - вывод в виде списка словарей

- Протестировать решение

- Проанализировать результат и предложить варианты улучшения

- Создать документацию

 - описание признаков
 - какая модель используется
 - как оценивается качество
 - инструкция по запуску (применению)

- Готовый проект предоставить в виде репозитория на гитхабе

- Демо приложение streamlit (**опционально**)


**Результат:**

Репозиторий на гитхабе должен содержать:

 - тетрадь jupyter notebook с описанием
  - подготовка признаков
  - обучение модели
  - тестирование

- описание проекта и инструкция по использованию в файле README.md

- список зависимостей в файле requirements.txt

**Стек:**

python, pandas, scikit-learn, NLP, transformers


###**Загрузка данных**

In [1]:
#  !pip freeze > requirements.txt

In [2]:
#проект запускаю в colab
import sys
ENV_COLAB = 'google.colab' in sys.modules

if ENV_COLAB:
    #!pip install streamlit >> None
    #!pip install pytorch-transformers >> None
    !pip install transformers >> None
    #!pip install pyinflect >> None
    #!pip install sentence_splitter >> None
    #!pip install contractions >> None
    !pip install sentence-transformers >>None
    !pip install datasets >>None
    !pip install nlpaug >> None


    print('Environment: Google Colab')

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.
Environment: Google Colab


In [3]:
!pip install streamlit

In [4]:
import pandas as pd
import numpy as np
import random
import streamlit as st

import sys
import re
import os

import torch
#import logging

import nlpaug.augmenter.word as naw
from nlpaug.util import Action
from datasets import load_dataset

from sentence_transformers import (
    SentenceTransformer,
    util,
    models,
    InputExample,
    losses,
    evaluation
)
from torch.utils.data import DataLoader
#import psycopg2
#import gensim
#from gensim.utils import simple_preprocess
#import gensim.downloader as api
#from gensim.models import Word2Vec
#from psycopg2 import Error
#from pathlib import Path
from tqdm import tqdm
# кроссвалидация
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

RANDOM_STATE = 7

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:
%cd /content/drive/My Drive/Colab Notebooks/

/content/drive/My Drive/Colab Notebooks


In [7]:
df1 = pd.read_csv('Примерное написание.csv')
df1.head()

,school_id,name
0,1836,"ООО ""Триумф"""
1,1836,"Москва, СК ""Триумф"""
2,610,"СШОР ""Надежда Губернии"
3,610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе..."
4,609,"""СШ ""Гвоздика"""


In [8]:
df2 = pd.read_csv('Школы.csv')
df2.head()

,school_id,name,region
0,1,Авангард,Московская область
1,2,Авангард,Ямало-Ненецкий АО
2,3,Авиатор,Республика Татарстан
3,4,Аврора,Санкт-Петербург
4,5,Ice Dream / Айс Дрим,Санкт-Петербург


In [9]:
#Объединим все данные в один датафрейм
df = df1 \
    .merge(df2, how='left', on='school_id')


In [10]:
df.head()

,school_id,name_x,name_y,region
0,1836,"ООО ""Триумф""","ООО ""Триумф""",Москва
1,1836,"Москва, СК ""Триумф""","ООО ""Триумф""",Москва
2,610,"СШОР ""Надежда Губернии","СШОР ""Надежда Губернии",Саратовская область
3,610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе...","СШОР ""Надежда Губернии",Саратовская область
4,609,"""СШ ""Гвоздика""","""СШ ""Гвоздика""",Удмуртская республика


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 895 entries, 0 to 894
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   school_id  895 non-null    int64 
 1   name_x     895 non-null    object
 2   name_y     895 non-null    object
 3   region     895 non-null    object
dtypes: int64(1), object(3)
memory usage: 28.1+ KB


In [12]:
df.shape

(895, 4)

In [13]:
df = df.rename(columns={'name_x': 'alternate_name'})
df = df.rename(columns={'name_y': 'name'})
df.head()

,school_id,alternate_name,name,region
0,1836,"ООО ""Триумф""","ООО ""Триумф""",Москва
1,1836,"Москва, СК ""Триумф""","ООО ""Триумф""",Москва
2,610,"СШОР ""Надежда Губернии","СШОР ""Надежда Губернии",Саратовская область
3,610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе...","СШОР ""Надежда Губернии",Саратовская область
4,609,"""СШ ""Гвоздика""","""СШ ""Гвоздика""",Удмуртская республика


In [14]:
df['alternate_name'] = (df['alternate_name']
                        .replace(r'\(.+\)', ' ',regex=True)
                        .replace(r'\[^А-Яа-яёЁА-Za-z\s]',' ',regex=True)
                        .replace(r'\s+', ' ',regex=True)
                        .str.strip())

In [15]:
df.head()

,school_id,alternate_name,name,region
0,1836,"ООО ""Триумф""","ООО ""Триумф""",Москва
1,1836,"Москва, СК ""Триумф""","ООО ""Триумф""",Москва
2,610,"СШОР ""Надежда Губернии","СШОР ""Надежда Губернии",Саратовская область
3,610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе...","СШОР ""Надежда Губернии",Саратовская область
4,609,"""СШ ""Гвоздика""","""СШ ""Гвоздика""",Удмуртская республика


In [16]:
df

,school_id,alternate_name,name,region
0,1836,"ООО ""Триумф""","ООО ""Триумф""",Москва
1,1836,"Москва, СК ""Триумф""","ООО ""Триумф""",Москва
2,610,"СШОР ""Надежда Губернии","СШОР ""Надежда Губернии",Саратовская область
3,610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе...","СШОР ""Надежда Губернии",Саратовская область
4,609,"""СШ ""Гвоздика""","""СШ ""Гвоздика""",Удмуртская республика
...,...,...,...,...
890,3,"Республика Татарстан, СШОР ФСО Авиатор",Авиатор,Республика Татарстан
891,3,"СШОР ФСО Авиатор, Республика Татарстан",Авиатор,Республика Татарстан
892,3,"Республика Татарстан, МБУ ДО СШОР «ФСО ""Авиатор""»",Авиатор,Республика Татарстан
893,2,"ЯНАО, СШ ""Авангард""",Авангард,Ямало-Ненецкий АО


In [17]:
#df.explode('alternate_name')

**Не получается использовать NLPaug**

In [18]:
#aug = naw.SpellingAug()
#df['Augmented']= aug.augment(df['alternate_name'], n=15)


In [19]:
train, test = train_test_split(df, test_size=0.25, random_state=RANDOM_STATE )

In [20]:
train.shape

(671, 4)

In [21]:
test.shape

(224, 4)

In [22]:
df['example'] = (df[['name', 'alternate_name']]
                        .apply(lambda x:InputExample(texts=list(x)),
                               axis = 1))

train_examples = df['example'].to_list()

In [23]:
model = SentenceTransformer('sentence-transformers/LaBSE')

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [24]:
df = df[['name', 'alternate_name']]
df

,name,alternate_name
0,"ООО ""Триумф""","ООО ""Триумф"""
1,"ООО ""Триумф""","Москва, СК ""Триумф"""
2,"СШОР ""Надежда Губернии","СШОР ""Надежда Губернии"
3,"СШОР ""Надежда Губернии","Саратовская область, ГБУСО ""СШОР ""Надежда Губе..."
4,"""СШ ""Гвоздика""","""СШ ""Гвоздика"""
...,...,...
890,Авиатор,"Республика Татарстан, СШОР ФСО Авиатор"
891,Авиатор,"СШОР ФСО Авиатор, Республика Татарстан"
892,Авиатор,"Республика Татарстан, МБУ ДО СШОР «ФСО ""Авиатор""»"
893,Авангард,"ЯНАО, СШ ""Авангард"""


In [25]:
corpus = model.encode(df.name.values)
corpus

array([[-6.1043492e-03, -2.5760822e-02, -1.9765386e-02, ...,
        -3.1093400e-02, -2.2926178e-02,  1.9848065e-02],
       [-6.1043408e-03, -2.5760796e-02, -1.9765377e-02, ...,
        -3.1093374e-02, -2.2926174e-02,  1.9848097e-02],
       [-4.9904622e-02,  2.2614328e-03,  1.2164296e-02, ...,
        -3.7194684e-02,  3.2219917e-02, -3.8257841e-02],
       ...,
       [-4.9950466e-03, -4.0322602e-02, -7.1538396e-02, ...,
         2.8259994e-02, -4.9056351e-02, -2.4897166e-02],
       [-4.9575530e-02,  1.3401947e-02,  1.0818915e-02, ...,
         6.2392966e-05, -7.4685208e-02, -6.2245138e-02],
       [-4.9575530e-02,  1.3401947e-02,  1.0818915e-02, ...,
         6.2392966e-05, -7.4685208e-02, -6.2245138e-02]], dtype=float32)

In [26]:
query = model.encode(test.alternate_name.values)
query

array([[ 0.01158053,  0.01149339, -0.04135942, ..., -0.02970115,
         0.01941699,  0.01319497],
       [ 0.05310474, -0.04124671, -0.01333536, ...,  0.03629863,
         0.00429573,  0.02882786],
       [ 0.01744296, -0.05539991,  0.0114769 , ...,  0.04205042,
        -0.00771083,  0.00158287],
       ...,
       [ 0.0526408 , -0.03044167, -0.04392361, ..., -0.02883205,
         0.06656347,  0.004614  ],
       [-0.01905926, -0.07355785, -0.0763818 , ...,  0.03621225,
        -0.07894833, -0.02900467],
       [-0.03461667, -0.00716475,  0.03642526, ..., -0.01675962,
         0.02588915, -0.0152944 ]], dtype=float32)

In [27]:
test.alternate_name

807              МБУ СШОР №13 Волгарь, Самарская область
779        Тверская область, МБУ ДО "СШ им. И. Я Гусева"
514                       Тульская область, ГУ ТО ОКСШОР
251           Оренбургская область, ДЮСШ с/к "Юбилейный"
259         Республика Татарстан, ГБУ ДО РСШОР по ФККиШТ
                             ...                        
418    Нижегородская область, МБУ СШ ДС "Северная Зве...
375                    Пермский край, КГБУ "СШОР "Старт"
827          Брянская область, СШ по ФКК ГАУ СК "Брянск"
20                                          ГУ ТО ОКСШОР
297                   Нижегородская область, СШОР по ЛВС
Name: alternate_name, Length: 224, dtype: object

In [28]:
search_result = util.semantic_search(query, corpus, top_k= 1)
search_result

[[{'corpus_id': 189, 'score': 0.6336479187011719}],
 [{'corpus_id': 2, 'score': 0.4861260950565338}],
 [{'corpus_id': 80, 'score': 0.521275520324707}],
 [{'corpus_id': 300, 'score': 0.5518448352813721}],
 [{'corpus_id': 14, 'score': 0.47805434465408325}],
 [{'corpus_id': 224, 'score': 0.737573504447937}],
 [{'corpus_id': 224, 'score': 0.6056820154190063}],
 [{'corpus_id': 80, 'score': 0.5984636545181274}],
 [{'corpus_id': 310, 'score': 0.4622563421726227}],
 [{'corpus_id': 80, 'score': 0.5906475782394409}],
 [{'corpus_id': 572, 'score': 0.6565598249435425}],
 [{'corpus_id': 80, 'score': 0.5656047463417053}],
 [{'corpus_id': 812, 'score': 0.8416454792022705}],
 [{'corpus_id': 749, 'score': 0.6204273700714111}],
 [{'corpus_id': 672, 'score': 0.6020256876945496}],
 [{'corpus_id': 300, 'score': 0.6372848749160767}],
 [{'corpus_id': 533, 'score': 0.5657902956008911}],
 [{'corpus_id': 533, 'score': 0.6223975419998169}],
 [{'corpus_id': 791, 'score': 0.6253629922866821}],
 [{'corpus_id': 99, 

In [29]:
test['canditate_idx'] = [x [0]['corpus_id'] for x in search_result]
test

,school_id,alternate_name,name,region,canditate_idx
807,33,"МБУ СШОР №13 Волгарь, Самарская область",Волгарь,Самарская область,189
779,41,"Тверская область, МБУ ДО ""СШ им. И. Я Гусева""",Гусева,Тверская область,2
514,117,"Тульская область, ГУ ТО ОКСШОР",ОКСШОР,Тульская область,80
251,203,"Оренбургская область, ДЮСШ с/к ""Юбилейный""",СШОР № 1,Оренбургская область,300
259,201,"Республика Татарстан, ГБУ ДО РСШОР по ФККиШТ",СШОР по ФККиШТ,Республика Татарстан,14
...,...,...,...,...,...
418,150,"Нижегородская область, МБУ СШ ДС ""Северная Зве...",Северная Звезда,Нижегородская область,418
375,168,"Пермский край, КГБУ ""СШОР ""Старт""",Старт,Пермский край,30
827,26,"Брянская область, СШ по ФКК ГАУ СК ""Брянск""",Брянск,Брянская область,827
20,295,ГУ ТО ОКСШОР,ГУ ТО ОКСШОР,Тульская область,20


In [30]:
test['canditate_name']= df.name.values[test.canditate_idx.values]
test

,school_id,alternate_name,name,region,canditate_idx,canditate_name
807,33,"МБУ СШОР №13 Волгарь, Самарская область",Волгарь,Самарская область,189,СШОР №3
779,41,"Тверская область, МБУ ДО ""СШ им. И. Я Гусева""",Гусева,Тверская область,2,"СШОР ""Надежда Губернии"
514,117,"Тульская область, ГУ ТО ОКСШОР",ОКСШОР,Тульская область,80,ЦФКСиЗ Московского района
251,203,"Оренбургская область, ДЮСШ с/к ""Юбилейный""",СШОР № 1,Оренбургская область,300,СШОР Колпинского района
259,201,"Республика Татарстан, ГБУ ДО РСШОР по ФККиШТ",СШОР по ФККиШТ,Республика Татарстан,14,"РФСОО ""ФФККРК"""
...,...,...,...,...,...,...
418,150,"Нижегородская область, МБУ СШ ДС ""Северная Зве...",Северная Звезда,Нижегородская область,418,Северная Звезда
375,168,"Пермский край, КГБУ ""СШОР ""Старт""",Старт,Пермский край,30,"СШ (ОРК) филиала ФАУ МО РФ ЦСКА (СКА, г. Санкт..."
827,26,"Брянская область, СШ по ФКК ГАУ СК ""Брянск""",Брянск,Брянская область,827,Брянск
20,295,ГУ ТО ОКСШОР,ГУ ТО ОКСШОР,Тульская область,20,ГУ ТО ОКСШОР


In [31]:
(test.name == test.canditate_name).sum()/test.shape[0]

0.30357142857142855

Model не дообучил, так как не получается использовать NLPAUg

In [32]:
df['example'] = (df[['name', 'alternate_name']]
                        .apply(lambda x:InputExample(texts=list(x)),
                               axis = 1))

train_examples = df['example'].to_list()

In [33]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.MultipleNegativesRankingLoss(model=model)

In [35]:
#model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=3)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
make_example = lambda x: InputExample(texts =[x['name'], x['alternate_name']])
train_examples = data['example'].to_list()

In [ ]:
examples = train [['name', 'alternate_name']].apply(make_example, axis= 1).values


In [ ]:
labse = SentenceTransformer('sentence-transformers/LaBSE')

In [ ]:
train_dataloader = DataLoader(examples, shuffle=True, batch_size=8)
train_loss = losses.MultipleNegativesRankingLoss(model=labse)

In [ ]:
labse.fit(train_objectives=[(train_dataloader, train_loss)], epochs=3)